# In this Notebook we are going to evaluate the `GEMMA 2B models(fine tuned on 3k data and 400 data)` & `DeepSeek Coder 1.3B Models(Fine Tuned on two different senarios)` using `SCAR BLEU`, `BLEU SCORE`, `EM RATIO`, `M.E.T.E.O.R` to check which one performes better.

High Level Overview of this Notebook 👇

1. Install Packages
2. Import Libraries
3. Load the Test Dataset
4. Load the Base Model (`GEMMA-2B from Google HuggingFace`)
5. Inference on the Test dataset using the Base Model
6. Load the Fine Tuned GEMMA-2B(400 rows) Model
7. Inference on the Test dataset using the Fine Tuned Model
8. Load the Fine Tuned GEMMA-2B(3000 rows) Model
9. Inference on the Test dataset using the Fine Tuned Model
10. Import the Base and two different versions of (`DeepSeek Coder Model`).
11. Calculate `SCAR BLEU SCORE`, `BLEU SCORE` & `EM RATIO` for the 6 Models
12. Compare each Models.


 # 1.Installing Libraries

In [ ]:
!pip3 install -q -U bitsandbytes==0.42.0
!pip3 install -q -U peft==0.8.2
!pip3 install -q -U trl==0.7.10
!pip3 install -q -U accelerate==0.27.1
!pip3 install -q -U datasets==2.17.0
!pip3 install -q -U transformers==4.38.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.7/279.7 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 23.1 MB/s eta 0:00:00


In [ ]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.3 MB/s eta 0:00:00


In [ ]:
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 1.3 MB/s eta 0:00:00


# 2. Importing Libraries

In [ ]:
import os
import transformers
import torch
from google.colab import userdata
from datasets import load_dataset
from trl import SFTTrainer
from peft import LoraConfig
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig, GemmaTokenizer
import evaluate
from tqdm import tqdm

# 3.Load the Test Dataset

In [ ]:
test_data = load_dataset('aryachakraborty/Sample_Test_DataSet')

Generating train split:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
test_data = test_data['train'].to_pandas()

In [ ]:
test_data.head()

,text,prompt,__index_level_0__
0,Below are sql tables schemas paired with instr...,Below are sql tables schemas paired with instr...,17864
1,Below are sql tables schemas paired with instr...,Below are sql tables schemas paired with instr...,748
2,Below are sql tables schemas paired with instr...,Below are sql tables schemas paired with instr...,2565
3,Below are sql tables schemas paired with instr...,Below are sql tables schemas paired with instr...,6150
4,Below are sql tables schemas paired with instr...,Below are sql tables schemas paired with instr...,18487


In [ ]:
test_data.drop(columns=['__index_level_0__'],inplace=True)
test_data.head()

,text,prompt
0,Below are sql tables schemas paired with instr...,Below are sql tables schemas paired with instr...
1,Below are sql tables schemas paired with instr...,Below are sql tables schemas paired with instr...
2,Below are sql tables schemas paired with instr...,Below are sql tables schemas paired with instr...
3,Below are sql tables schemas paired with instr...,Below are sql tables schemas paired with instr...
4,Below are sql tables schemas paired with instr...,Below are sql tables schemas paired with instr...


Printing one sample from prompt and text column to see the difference

In [ ]:
test_data['prompt'] = test_data['prompt'] + ' ### Response:'

In [ ]:
print(test_data['prompt'][50])
print('++'*100)
print(test_data['text'][50])

Below are sql tables schemas paired with instruction that describes a task. Using valid SQLite, write a response that appropriately completes the request for the provided tables. ### Instruction: What are the heights of perpetrators in descending order of the number of people they injured? ### Input: CREATE TABLE people (
    Height VARCHAR,
    People_ID VARCHAR
)

CREATE TABLE perpetrator (
    People_ID VARCHAR,
    Injured VARCHAR
). ### Response:
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Below are sql tables schemas paired with instruction that describes a task. Using valid SQLite, write a response that appropriately completes the request for the provided tables. ### Instruction: What are the heights of perpetrators in descending order of the number of people they injured? ### Input: CREATE TABLE people (
    Height VARCHAR,

# 4. Loading the Base Model

In [ ]:
os.environ['HF_TOKEN'] = userdata.get('HF_TOKEN')

In [ ]:
model_id = "google/gemma-2b-it"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)# Add double quantization=True


tokenizer = AutoTokenizer.from_pretrained(model_id, token=os.environ['HF_TOKEN'])
model = AutoModelForCausalLM.from_pretrained(model_id,
                                             quantization_config=bnb_config,
                                             device_map={"":0},
                                             token=os.environ['HF_TOKEN'])
tokenizer.pad_token=tokenizer.eos_token
tokenizer.padding_side="right"

tokenizer_config.json:   0%|          | 0.00/2.16k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/888 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

# 5. Inference on the Test dataset using the Base Model and saving the output in a new column called `base_model_output`

In [ ]:
device = "cuda:0"
def generate_output(row):
    inputs = tokenizer(row['prompt'], return_tensors="pt").to(device)
    outputs = model.generate(**inputs,max_new_tokens=30)
    generated_text =  tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_text

test_data['base_model_output'] = test_data.apply(lambda row: generate_output(row),axis=1)

print(test_data.head())

In [ ]:
test_data.head()

,text,prompt,base_model_output
0,Below are sql tables schemas paired with instr...,Below are sql tables schemas paired with instr...,Below are sql tables schemas paired with instr...
1,Below are sql tables schemas paired with instr...,Below are sql tables schemas paired with instr...,Below are sql tables schemas paired with instr...
2,Below are sql tables schemas paired with instr...,Below are sql tables schemas paired with instr...,Below are sql tables schemas paired with instr...
3,Below are sql tables schemas paired with instr...,Below are sql tables schemas paired with instr...,Below are sql tables schemas paired with instr...
4,Below are sql tables schemas paired with instr...,Below are sql tables schemas paired with instr...,Below are sql tables schemas paired with instr...


In [ ]:
print(test_data['text'][29])
print('++'*100)
print(test_data['base_model_output'][29])

Below are sql tables schemas paired with instruction that describes a task. Using valid SQLite, write a response that appropriately completes the request for the provided tables. ### Instruction: What is the most gold medals that a team ranked higher than 6, have 1 silver medal, and more than 4 total medals have? ### Input: CREATE TABLE table_54285 (
    "Rank" real,
    "Nation" text,
    "Gold" real,
    "Silver" real,
    "Bronze" real,
    "Total" real
) ### Response: SELECT MAX("Gold") FROM table_54285 WHERE "Rank" < '6' AND "Silver" = '1' AND "Total" > '4'
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Below are sql tables schemas paired with instruction that describes a task. Using valid SQLite, write a response that appropriately completes the request for the provided tables. ### Instruction: What is the most gold medals that 

# 6. Loading the Fine tuned model which was fine tuned on 400 rows

In [ ]:
model_id = "aryachakraborty/GEMMA-2B-NL-SQL"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer2 = AutoTokenizer.from_pretrained(model_id, token=os.environ['HF_TOKEN'])
model2 = AutoModelForCausalLM.from_pretrained(model_id,
                                             quantization_config=bnb_config,
                                             device_map={"":0},
                                             token=os.environ['HF_TOKEN'])

tokenizer_config.json:   0%|          | 0.00/2.16k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/522 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

# 7. Inference on the Test dataset using the Fine Tuned Model(on 400 rows) and saving the output in a new column called `FT_model_output_400`

In [ ]:
device = "cuda:0"
def generate_output2(row):
    inputs = tokenizer2(row['prompt'], return_tensors="pt").to(device)
    outputs = model2.generate(**inputs,max_new_tokens=30)
    generated_text =  tokenizer2.decode(outputs[0], skip_special_tokens=True)
    return generated_text

# Apply the function to each row of the 'template_without_output' column
test_data['FT_model_output_400'] = test_data.apply(lambda row: generate_output2(row),axis=1)

test_data.head()

,text,prompt,base_model_output,FT_model_output_400
0,Below are sql tables schemas paired with instr...,Below are sql tables schemas paired with instr...,Below are sql tables schemas paired with instr...,Below are sql tables schemas paired with instr...
1,Below are sql tables schemas paired with instr...,Below are sql tables schemas paired with instr...,Below are sql tables schemas paired with instr...,Below are sql tables schemas paired with instr...
2,Below are sql tables schemas paired with instr...,Below are sql tables schemas paired with instr...,Below are sql tables schemas paired with instr...,Below are sql tables schemas paired with instr...
3,Below are sql tables schemas paired with instr...,Below are sql tables schemas paired with instr...,Below are sql tables schemas paired with instr...,Below are sql tables schemas paired with instr...
4,Below are sql tables schemas paired with instr...,Below are sql tables schemas paired with instr...,Below are sql tables schemas paired with instr...,Below are sql tables schemas paired with instr...


In [ ]:
print(test_data['text'][0])
print('++'*100)
print(test_data['base_model_output'][0])
print('++'*100)
print(test_data['FT_model_output_400'][0])

Below are sql tables schemas paired with instruction that describes a task. Using valid SQLite, write a response that appropriately completes the request for the provided tables. ### Instruction: What is the highest crowd listed when the away side scores 4.9 (33)? ### Input: CREATE TABLE table_54182 (
    "Home team" text,
    "Home team score" text,
    "Away team" text,
    "Away team score" text,
    "Venue" text,
    "Crowd" real,
    "Date" text
) ### Response: SELECT MAX("Crowd") FROM table_54182 WHERE "Away team score" = '4.9 (33)'
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Below are sql tables schemas paired with instruction that describes a task. Using valid SQLite, write a response that appropriately completes the request for the provided tables. ### Instruction: What is the highest crowd listed when the away side scores

## So we can see that my fine tuned model is not working bad. Although we can't just have faith in it , it just have been fine tuned on 400 rows

# 8. Loading the Fine tuned model which was fine tuned on 3000 rows

In [ ]:
model_id = "aryachakraborty/Fine_Tuned_GEMMA-2B-3K-dataset"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer3 = AutoTokenizer.from_pretrained(model_id, token=os.environ['HF_TOKEN'])
model3 = AutoModelForCausalLM.from_pretrained(model_id,
                                             quantization_config=bnb_config,
                                             device_map={"":0},
                                             token=os.environ['HF_TOKEN'])

tokenizer_config.json:   0%|          | 0.00/2.16k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/522 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

# 9. Inference on the Test dataset using the Fine Tuned Model(on 3000 rows) and saving the output in a new column called `FT_model_output_3000`

In [ ]:
device = "cuda:0"
def generate_output3(row):
    inputs = tokenizer3(row['prompt'], return_tensors="pt").to(device)
    outputs = model3.generate(**inputs,max_new_tokens=30)
    generated_text =  tokenizer3.decode(outputs[0], skip_special_tokens=True)
    return generated_text

# Apply the function to each row of the 'template_without_output' column
test_data['FT_model_output_3000'] = test_data.apply(lambda row: generate_output2(row),axis=1)

test_data.head()

,text,prompt,base_model_output,FT_model_output_400,FT_model_output_3000
0,Below are sql tables schemas paired with instr...,Below are sql tables schemas paired with instr...,Below are sql tables schemas paired with instr...,Below are sql tables schemas paired with instr...,Below are sql tables schemas paired with instr...
1,Below are sql tables schemas paired with instr...,Below are sql tables schemas paired with instr...,Below are sql tables schemas paired with instr...,Below are sql tables schemas paired with instr...,Below are sql tables schemas paired with instr...
2,Below are sql tables schemas paired with instr...,Below are sql tables schemas paired with instr...,Below are sql tables schemas paired with instr...,Below are sql tables schemas paired with instr...,Below are sql tables schemas paired with instr...
3,Below are sql tables schemas paired with instr...,Below are sql tables schemas paired with instr...,Below are sql tables schemas paired with instr...,Below are sql tables schemas paired with instr...,Below are sql tables schemas paired with instr...
4,Below are sql tables schemas paired with instr...,Below are sql tables schemas paired with instr...,Below are sql tables schemas paired with instr...,Below are sql tables schemas paired with instr...,Below are sql tables schemas paired with instr...


In [ ]:
print(test_data['text'][50])
print('++'*100)
print(test_data['base_model_output'][50])
print('++'*100)
print(test_data['FT_model_output_400'][50])
print('++'*100)
print(test_data['FT_model_output_3000'][50])

Below are sql tables schemas paired with instruction that describes a task. Using valid SQLite, write a response that appropriately completes the request for the provided tables. ### Instruction: What are the heights of perpetrators in descending order of the number of people they injured? ### Input: CREATE TABLE people (
    Height VARCHAR,
    People_ID VARCHAR
)

CREATE TABLE perpetrator (
    People_ID VARCHAR,
    Injured VARCHAR
) ### Response: SELECT T1.Height FROM people AS T1 JOIN perpetrator AS T2 ON T1.People_ID = T2.People_ID ORDER BY T2.Injured DESC
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Below are sql tables schemas paired with instruction that describes a task. Using valid SQLite, write a response that appropriately completes the request for the provided tables. ### Instruction: What are the heights of perpetrato

# Loading the `Deepseek Coder V1 Model`

In [ ]:
import pandas as pd
test_data2 = pd.read_csv('/content/llm_reponse.csv')

test_data2['Deepseek_Prompt'] = test_data2['prompt'].apply(lambda x: "You are an AI programming assistant, utilizing the Deepseek Coder model, developed by arya chakraborty, and your task is to convert natural language to sql queries. For politically sensitive questions, security and privacy issues, and other non-computer science questions, you will refuse to answer.\n" + x)

test_data2.head(2)

,text,prompt,base_model_output,FT_model_output_400,FT_model_output_3000,Deepseek_Prompt
0,"SELECT MAX(""Crowd"") FROM table_54182 WHERE ""Aw...",Below are sql tables schemas paired with instr...,The highest crowd listed when the away side sc...,"SELECT MAX(""Crowd"") FROM table_54182 WHERE ""Aw...","SELECT MAX(""Crowd"") FROM table_54182 WHERE ""Aw...","You are an AI programming assistant, utilizing..."
1,"SELECT ""Surface"" FROM table_25435 WHERE ""Again...",Below are sql tables schemas paired with instr...,```sql\nCREATE TABLE surface_against_monaco (\...,"SELECT ""Surface"" FROM table_25435 WHERE ""Again...","SELECT ""Surface"" FROM table_25435 WHERE ""Again...","You are an AI programming assistant, utilizing..."


In [ ]:
model_id = "aryachakraborty/DeepSeek_1.3B_Fine_Tuned"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer4 = AutoTokenizer.from_pretrained(model_id, token=os.environ['HF_TOKEN'])
model4 = AutoModelForCausalLM.from_pretrained(model_id,
                                             quantization_config=bnb_config,
                                             device_map={"":0},
                                             token=os.environ['HF_TOKEN'])

tokenizer_config.json:   0%|          | 0.00/5.31k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.37M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/329 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/748 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.69G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

#Inference on the Test dataset using the Fine Tuned DeepSeek Coder V1 Model and saving the output in a new column called `FT_DeepSeek_V1`

In [ ]:
device = "cuda:0"
def generate_output4(row):
    inputs = tokenizer4(row['Deepseek_Prompt'], return_tensors="pt").to(device)
    outputs = model4.generate(**inputs,max_new_tokens=30)
    generated_text =  tokenizer4.decode(outputs[0], skip_special_tokens=True)
    return generated_text

# Apply the function to each row of the 'template_without_output' column
test_data2['FT_DeepSeek_V1'] = test_data2.apply(lambda row: generate_output4(row),axis=1)


In [ ]:
test_data2["FT_DeepSeek_V1"] = test_data2["FT_DeepSeek_V1"].str.split("Response:").str[1].str.strip()
test_data2.head()

,text,prompt,base_model_output,FT_model_output_400,FT_model_output_3000,Deepseek_Prompt,FT_DeepSeek_V1
0,"SELECT MAX(""Crowd"") FROM table_54182 WHERE ""Aw...",Below are sql tables schemas paired with instr...,The highest crowd listed when the away side sc...,"SELECT MAX(""Crowd"") FROM table_54182 WHERE ""Aw...","SELECT MAX(""Crowd"") FROM table_54182 WHERE ""Aw...","You are an AI programming assistant, utilizing...","SELECT MAX(""Crowd"") FROM table_54182 WHERE ""Aw..."
1,"SELECT ""Surface"" FROM table_25435 WHERE ""Again...",Below are sql tables schemas paired with instr...,```sql\nCREATE TABLE surface_against_monaco (\...,"SELECT ""Surface"" FROM table_25435 WHERE ""Again...","SELECT ""Surface"" FROM table_25435 WHERE ""Again...","You are an AI programming assistant, utilizing...","SELECT ""Surface"" FROM table_25435 WHERE ""Again..."
2,SELECT MIN(floor_exercise) FROM table_name_58 ...,Below are sql tables schemas paired with instr...,**Floor Exercise:**\n- Name: Dumbbell Bicep Cu...,SELECT floor_exercise FROM table_name_58 WHERE...,SELECT floor_exercise FROM table_name_58 WHERE...,"You are an AI programming assistant, utilizing...",SELECT parallel_bars FROM table_name_58 WHERE ...
3,"SELECT COUNT(""Production code"") FROM table_265...",Below are sql tables schemas paired with instr...,12\n\n```sql\nCREATE TABLE production_codes (\...,"SELECT COUNT(""Production code"") FROM table_265...","SELECT COUNT(""Production code"") FROM table_265...","You are an AI programming assistant, utilizing...","SELECT COUNT(*) FROM table_26539 WHERE ""Origin..."
4,"SELECT ""Term in office"" FROM table_71587 WHERE...",Below are sql tables schemas paired with instr...,The term in office of Hon. Ralph Hunt was not ...,"SELECT ""Term in office"" FROM table_71587 WHERE...","SELECT ""Term in office"" FROM table_71587 WHERE...","You are an AI programming assistant, utilizing...","SELECT ""Term in office"" FROM table_71587 WHERE..."


In [ ]:
print(test_data2['text'][24])
print('++'*100)
print(test_data2['base_model_output'][24])
print('++'*100)
print(test_data2['FT_model_output_400'][24])
print('++'*100)
print(test_data2['FT_model_output_3000'][24])
print('++'*100)
print(test_data2['FT_DeepSeek_V1'][24])

SELECT AVG("Played") FROM table_42931 WHERE "Against" < '15'
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
```sql
SELECT AVG(Points) AS avg_played
FROM table_42931
WHERE Played < 15;
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
SELECT AVG("Played") FROM table_42931 WHERE "Against" < '15' GROUP BY "Played" ORDER BY "
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
SELECT AVG("Played") FROM table_42931 WHERE "Against" < '15' GROUP BY "Played" ORDER BY "
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

# Loading the `Base Deepseek Coder Model 1.3B`

In [ ]:
model_id = "deepseek-ai/deepseek-coder-1.3b-instruct"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer5 = AutoTokenizer.from_pretrained(model_id, token=os.environ['HF_TOKEN'])
model5 = AutoModelForCausalLM.from_pretrained(model_id,
                                             quantization_config=bnb_config,
                                             device_map={"":0},
                                             token=os.environ['HF_TOKEN'])

tokenizer_config.json:   0%|          | 0.00/1.87k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.37M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/631 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.69G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

# Inference on the Test dataset using the `Base DeepSeek Coder Model` and saving the output in a new column called `Base_DeepSeek_Coder_model`

In [ ]:
device = "cuda:0"
def generate_output5(row):
    inputs = tokenizer5(row['Deepseek_Prompt'], return_tensors="pt").to(device)
    outputs = model5.generate(**inputs,max_new_tokens=30)
    generated_text =  tokenizer5.decode(outputs[0], skip_special_tokens=True)
    return generated_text

# Apply the function to each row of the 'template_without_output' column
test_data2['Base_DeepSeek'] = test_data2.apply(lambda row: generate_output5(row),axis=1)

In [ ]:
test_data2["Base_DeepSeek"] = test_data2["Base_DeepSeek"].str.split("Response:").str[1].str.strip()
test_data2.head()

,text,prompt,base_model_output,FT_model_output_400,FT_model_output_3000,Deepseek_Prompt,FT_DeepSeek_V1,Base_DeepSeek
0,"SELECT MAX(""Crowd"") FROM table_54182 WHERE ""Aw...",Below are sql tables schemas paired with instr...,The highest crowd listed when the away side sc...,"SELECT MAX(""Crowd"") FROM table_54182 WHERE ""Aw...","SELECT MAX(""Crowd"") FROM table_54182 WHERE ""Aw...","You are an AI programming assistant, utilizing...","SELECT MAX(""Crowd"") FROM table_54182 WHERE ""Aw...","SELECT MAX(Crowd) FROM table_54182 WHERE ""Away..."
1,"SELECT ""Surface"" FROM table_25435 WHERE ""Again...",Below are sql tables schemas paired with instr...,```sql\nCREATE TABLE surface_against_monaco (\...,"SELECT ""Surface"" FROM table_25435 WHERE ""Again...","SELECT ""Surface"" FROM table_25435 WHERE ""Again...","You are an AI programming assistant, utilizing...","SELECT ""Surface"" FROM table_25435 WHERE ""Again...",SELECT * FROM table_25435 WHERE Surface = 'Mon...
2,SELECT MIN(floor_exercise) FROM table_name_58 ...,Below are sql tables schemas paired with instr...,**Floor Exercise:**\n- Name: Dumbbell Bicep Cu...,SELECT floor_exercise FROM table_name_58 WHERE...,SELECT floor_exercise FROM table_name_58 WHERE...,"You are an AI programming assistant, utilizing...",SELECT parallel_bars FROM table_name_58 WHERE ...,SELECT parallel_bars\nFROM table_name_58\nWHER...
3,"SELECT COUNT(""Production code"") FROM table_265...",Below are sql tables schemas paired with instr...,12\n\n```sql\nCREATE TABLE production_codes (\...,"SELECT COUNT(""Production code"") FROM table_265...","SELECT COUNT(""Production code"") FROM table_265...","You are an AI programming assistant, utilizing...","SELECT COUNT(*) FROM table_26539 WHERE ""Origin...","SELECT COUNT(*) FROM table_26539 WHERE ""Origin..."
4,"SELECT ""Term in office"" FROM table_71587 WHERE...",Below are sql tables schemas paired with instr...,The term in office of Hon. Ralph Hunt was not ...,"SELECT ""Term in office"" FROM table_71587 WHERE...","SELECT ""Term in office"" FROM table_71587 WHERE...","You are an AI programming assistant, utilizing...","SELECT ""Term in office"" FROM table_71587 WHERE...","SELECT ""Term in office"" FROM table_71587 WHERE..."


In [ ]:
print(test_data2['text'][24])
print('++'*100)
print(test_data2['base_model_output'][24])
print('++'*100)
print(test_data2['FT_model_output_400'][24])
print('++'*100)
print(test_data2['FT_model_output_3000'][24])
print('++'*100)
print(test_data2['Base_DeepSeek'][24])
print('++'*100)
print(test_data2['FT_DeepSeek_V1'][24])

SELECT AVG("Played") FROM table_42931 WHERE "Against" < '15'
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
```sql
SELECT AVG(Points) AS avg_played
FROM table_42931
WHERE Played < 15;
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
SELECT AVG("Played") FROM table_42931 WHERE "Against" < '15' GROUP BY "Played" ORDER BY "
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
SELECT AVG("Played") FROM table_42931 WHERE "Against" < '15' GROUP BY "Played" ORDER BY "
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

# Loading the `Deepseek Coder V2 Model`

In [ ]:
model_id = "aryachakraborty/DeepSeek-1.3B-IT-NL-SQL-V2"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer6 = AutoTokenizer.from_pretrained(model_id, token=os.environ['HF_TOKEN'])
model6 = AutoModelForCausalLM.from_pretrained(model_id,
                                             quantization_config=bnb_config,
                                             device_map={"":0},
                                             token=os.environ['HF_TOKEN'])

tokenizer_config.json:   0%|          | 0.00/5.31k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.37M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/329 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/748 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.69G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

# Inference on the Test dataset using the `Fine Tuned DeepSeek Coder Model V2`and saving the output in a new column called `FT_DeepSeek_Coder_model_V2`

In [ ]:
device = "cuda:0"
def generate_output6(row):
    inputs = tokenizer6(row['Deepseek_Prompt'], return_tensors="pt").to(device)
    outputs = model6.generate(**inputs,max_new_tokens=30)
    generated_text =  tokenizer6.decode(outputs[0], skip_special_tokens=True)
    return generated_text

# Apply the function to each row of the 'template_without_output' column
test_data2['FT_DeepSeek_Coder_model_V2'] = test_data2.apply(lambda row: generate_output6(row),axis=1)

In [ ]:
#test_data2["FT_DeepSeek_Coder_model_V2"] = test_data2["FT_DeepSeek_Coder_model_V2"].str.split("Response:").str[1].str.strip()
test_data2.head()

,text,prompt,base_model_output,FT_model_output_400,FT_model_output_3000,Deepseek_Prompt,FT_DeepSeek_V1,Base_DeepSeek,FT_DeepSeek_Coder_model_V2
0,"SELECT MAX(""Crowd"") FROM table_54182 WHERE ""Aw...",Below are sql tables schemas paired with instr...,The highest crowd listed when the away side sc...,"SELECT MAX(""Crowd"") FROM table_54182 WHERE ""Aw...","SELECT MAX(""Crowd"") FROM table_54182 WHERE ""Aw...","You are an AI programming assistant, utilizing...","SELECT MAX(""Crowd"") FROM table_54182 WHERE ""Aw...","SELECT MAX(Crowd) FROM table_54182 WHERE ""Away...","SELECT MAX(""Crowd"") FROM table_54182 WHERE ""Aw..."
1,"SELECT ""Surface"" FROM table_25435 WHERE ""Again...",Below are sql tables schemas paired with instr...,```sql\nCREATE TABLE surface_against_monaco (\...,"SELECT ""Surface"" FROM table_25435 WHERE ""Again...","SELECT ""Surface"" FROM table_25435 WHERE ""Again...","You are an AI programming assistant, utilizing...","SELECT ""Surface"" FROM table_25435 WHERE ""Again...",SELECT * FROM table_25435 WHERE Surface = 'Mon...,"SELECT ""Surface"" FROM table_25435 WHERE ""Again..."
2,SELECT MIN(floor_exercise) FROM table_name_58 ...,Below are sql tables schemas paired with instr...,**Floor Exercise:**\n- Name: Dumbbell Bicep Cu...,SELECT floor_exercise FROM table_name_58 WHERE...,SELECT floor_exercise FROM table_name_58 WHERE...,"You are an AI programming assistant, utilizing...",SELECT parallel_bars FROM table_name_58 WHERE ...,SELECT parallel_bars\nFROM table_name_58\nWHER...,SELECT floor_exercise FROM table_name_58 WHERE...
3,"SELECT COUNT(""Production code"") FROM table_265...",Below are sql tables schemas paired with instr...,12\n\n```sql\nCREATE TABLE production_codes (\...,"SELECT COUNT(""Production code"") FROM table_265...","SELECT COUNT(""Production code"") FROM table_265...","You are an AI programming assistant, utilizing...","SELECT COUNT(*) FROM table_26539 WHERE ""Origin...","SELECT COUNT(*) FROM table_26539 WHERE ""Origin...","SELECT COUNT(""Production code"") FROM table_265..."
4,"SELECT ""Term in office"" FROM table_71587 WHERE...",Below are sql tables schemas paired with instr...,The term in office of Hon. Ralph Hunt was not ...,"SELECT ""Term in office"" FROM table_71587 WHERE...","SELECT ""Term in office"" FROM table_71587 WHERE...","You are an AI programming assistant, utilizing...","SELECT ""Term in office"" FROM table_71587 WHERE...","SELECT ""Term in office"" FROM table_71587 WHERE...","SELECT ""Term in office"" FROM table_71587 WHERE..."


In [ ]:
print(test_data2['text'][24])
print('++'*100)
print(test_data2['base_model_output'][24])
print('++'*100)
print(test_data2['FT_model_output_400'][24])
print('++'*100)
print(test_data2['FT_model_output_3000'][24])
print('++'*100)
print(test_data2['Base_DeepSeek'][24])
print('++'*100)
print(test_data2['FT_DeepSeek_V1'][24])
print('++'*100)
print(test_data2['FT_DeepSeek_Coder_model_V2'][24])

SELECT AVG("Played") FROM table_42931 WHERE "Against" < '15'
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
```sql
SELECT AVG(Points) AS avg_played
FROM table_42931
WHERE Played < 15;
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
SELECT AVG("Played") FROM table_42931 WHERE "Against" < '15' GROUP BY "Played" ORDER BY "
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
SELECT AVG("Played") FROM table_42931 WHERE "Against" < '15' GROUP BY "Played" ORDER BY "
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

# Saving the dataframe as CSV for future refrence

In [ ]:
file_path_colab = '/content/outputs.csv'
test_data2.to_csv(file_path_colab,index=False)

# Evaluating the Models using `SACREBLEU BLEU SCORE`, `BLEU SCORE`, `EM RATIO`

In [ ]:
import pandas as pd
final_df = pd.read_csv('/content/outputs.csv')
final_df.head()

,text,prompt,base_model_output,FT_model_output_400,FT_model_output_3000,Deepseek_Prompt,FT_DeepSeek_V1,Base_DeepSeek,FT_DeepSeek_Coder_model_V2
0,"SELECT MAX(""Crowd"") FROM table_54182 WHERE ""Aw...",Below are sql tables schemas paired with instr...,The highest crowd listed when the away side sc...,"SELECT MAX(""Crowd"") FROM table_54182 WHERE ""Aw...","SELECT MAX(""Crowd"") FROM table_54182 WHERE ""Aw...","You are an AI programming assistant, utilizing...","SELECT MAX(""Crowd"") FROM table_54182 WHERE ""Aw...","SELECT MAX(Crowd) FROM table_54182 WHERE ""Away...","SELECT MAX(""Crowd"") FROM table_54182 WHERE ""Aw..."
1,"SELECT ""Surface"" FROM table_25435 WHERE ""Again...",Below are sql tables schemas paired with instr...,```sql\nCREATE TABLE surface_against_monaco (\...,"SELECT ""Surface"" FROM table_25435 WHERE ""Again...","SELECT ""Surface"" FROM table_25435 WHERE ""Again...","You are an AI programming assistant, utilizing...","SELECT ""Surface"" FROM table_25435 WHERE ""Again...",SELECT * FROM table_25435 WHERE Surface = 'Mon...,"SELECT ""Surface"" FROM table_25435 WHERE ""Again..."
2,SELECT MIN(floor_exercise) FROM table_name_58 ...,Below are sql tables schemas paired with instr...,**Floor Exercise:**\n- Name: Dumbbell Bicep Cu...,SELECT floor_exercise FROM table_name_58 WHERE...,SELECT floor_exercise FROM table_name_58 WHERE...,"You are an AI programming assistant, utilizing...",SELECT parallel_bars FROM table_name_58 WHERE ...,SELECT parallel_bars\nFROM table_name_58\nWHER...,SELECT floor_exercise FROM table_name_58 WHERE...
3,"SELECT COUNT(""Production code"") FROM table_265...",Below are sql tables schemas paired with instr...,12\n\n```sql\nCREATE TABLE production_codes (\...,"SELECT COUNT(""Production code"") FROM table_265...","SELECT COUNT(""Production code"") FROM table_265...","You are an AI programming assistant, utilizing...","SELECT COUNT(*) FROM table_26539 WHERE ""Origin...","SELECT COUNT(*) FROM table_26539 WHERE ""Origin...","SELECT COUNT(""Production code"") FROM table_265..."
4,"SELECT ""Term in office"" FROM table_71587 WHERE...",Below are sql tables schemas paired with instr...,The term in office of Hon. Ralph Hunt was not ...,"SELECT ""Term in office"" FROM table_71587 WHERE...","SELECT ""Term in office"" FROM table_71587 WHERE...","You are an AI programming assistant, utilizing...","SELECT ""Term in office"" FROM table_71587 WHERE...","SELECT ""Term in office"" FROM table_71587 WHERE...","SELECT ""Term in office"" FROM table_71587 WHERE..."


## SACREBLEU SCORE

- < 10: Almost useless
- 10-19: Hard to get the gist
- 20-29 :The gist is clear, but has significant grammatical errors
- 30-40: Understandable to good translations
- 40-50: High quality translations
- 50-60: Very high quality, adequate, and fluent translations
- greater than 60: Quality often better than human

In [ ]:
def calculate_sacrebleu_score(df,actual_col, pred_col):
  sacrebleu = evaluate.load("sacrebleu")
  sacrebleu_results=sacrebleu.compute(predictions=df[pred_col], references=df[actual_col])

  return  sacrebleu_results["score"]


print("SACR BLEU SCORE for Base GEMMA-2B Model :", calculate_sacrebleu_score(final_df,'text','base_model_output'))
print("SACR BLEU SCORE for Fine Tuned GEMMA-2B(400 rows) :", calculate_sacrebleu_score(final_df,'text','FT_model_output_400'))
print("SACR BLEU SCORE for Fine Tuned GEMMA-2B(3000 rows) :", calculate_sacrebleu_score(final_df,'text','FT_model_output_3000'))
print("SACR BLEU SCORE for Base DeepSeek Coder-1.3B :", calculate_sacrebleu_score(final_df,'text','Base_DeepSeek'))
print("SACR BLEU SCORE for Fine Tuned DeepSeek Coder-1.3B V1:", calculate_sacrebleu_score(final_df,'text','FT_DeepSeek_V1'))
print("SACR BLEU SCORE for Fine Tuned DeepSeek Coder-1.3B V2:", calculate_sacrebleu_score(final_df,'text','FT_DeepSeek_Coder_model_V2'))

SACR BLEU SCORE for Base GEMMA-2B Model : 3.5939636954910594
SACR BLEU SCORE for Fine Tuned GEMMA-2B(400 rows) : 70.38138191055378
SACR BLEU SCORE for Fine Tuned GEMMA-2B(3000 rows) : 70.38138191055378
SACR BLEU SCORE for Base DeepSeek Coder-1.3B : 41.79270016983948
SACR BLEU SCORE for Fine Tuned DeepSeek Coder-1.3B V1: 57.78160573416485
SACR BLEU SCORE for Fine Tuned DeepSeek Coder-1.3B V2: 70.3479609920323


## BLEU SCORE

In [ ]:
def calculate_bleu_score(df,actual_col, pred_col):
  bleu = evaluate.load("bleu")
  bleu_results=bleu.compute(predictions=df[pred_col], references=df[actual_col])

  return bleu_results['bleu']

print("BLEU SCORE for Base GEMMA-2B Model :", calculate_bleu_score(final_df,'text','base_model_output'))
print("BLEU SCORE for Fine Tuned GEMMA-2B(400 rows) :", calculate_bleu_score(final_df,'text','FT_model_output_400'))
print("BLEU SCORE for Fine Tuned GEMMA-2B(3000 rows):", calculate_bleu_score(final_df,'text','FT_model_output_3000'))
print("BLEU SCORE for Base DeepSeek Coder-1.3B:", calculate_bleu_score(final_df,'text','Base_DeepSeek'))
print("BLEU SCORE for Fine Tuned DeepSeek Coder-1.3B V1:", calculate_bleu_score(final_df,'text','FT_DeepSeek_V1'))
print("BLEU SCORE for Fine Tuned DeepSeek Coder-1.3B V2:", calculate_bleu_score(final_df,'text','FT_DeepSeek_Coder_model_V2'))

BLEU SCORE for Base GEMMA-2B Model : 0.035939636954910585
BLEU SCORE for Fine Tuned GEMMA-2B(400 rows) : 0.703813819105538
BLEU SCORE for Fine Tuned GEMMA-2B(3000 rows): 0.703813819105538
BLEU SCORE for Base DeepSeek Coder-1.3B: 0.41792700169839475
BLEU SCORE for Fine Tuned DeepSeek Coder-1.3B V1: 0.5778160573416482
BLEU SCORE for Fine Tuned DeepSeek Coder-1.3B V2: 0.7034796099203228


## Now we will Evaluate the Models using EM Ratio Method

In [ ]:
def calculate_exact_match_ratio(df,actual_col, pred_col):
    total_samples = len(df)
    exact_matches = 0

    for _, row in df.iterrows():
      if  actual_col in df.columns and pred_col in df.columns:
          actual_output = row[actual_col]
          model_output = row[pred_col]

          if actual_output == model_output:
              exact_matches += 1
      else:
            # Handle the case where the required columns do not exist
            print("Error: Required columns not found in DataFrame.")
            return None
    em_score = exact_matches / total_samples
    return em_score


print("Exact Match Ratio (EM) for GEMMA-2B Model :", calculate_exact_match_ratio(final_df,'text','base_model_output'))
print("Exact Match Ratio (EM) for Fine Tuned GEMMA-2B(400 rows) :", calculate_exact_match_ratio(final_df,'text','FT_model_output_400'))
print("Exact Match Ratio (EM) for Fine Tuned GEMMA-2B(3000 rows) :", calculate_exact_match_ratio(final_df,'text','FT_model_output_3000'))
print("Exact Match Ratio (EM) for Base DeepSeek Coder 1.3B :", calculate_exact_match_ratio(final_df,'text','Base_DeepSeek'))
print("Exact Match Ratio (EM) for Fine Tuned DeepSeek Coder 1.3B V1:", calculate_exact_match_ratio(final_df,'text','FT_DeepSeek_V1'))
print("Exact Match Ratio (EM) for Fine Tuned DeepSeek Coder 1.3B V2:", calculate_exact_match_ratio(final_df,'text','FT_DeepSeek_Coder_model_V2'))


Exact Match Ratio (EM) for GEMMA-2B Model : 0.0
Exact Match Ratio (EM) for Fine Tuned GEMMA-2B(400 rows) : 0.14
Exact Match Ratio (EM) for Fine Tuned GEMMA-2B(3000 rows) : 0.14
Exact Match Ratio (EM) for Base DeepSeek Coder 1.3B : 0.01
Exact Match Ratio (EM) for Fine Tuned DeepSeek Coder 1.3B V1: 0.0
Exact Match Ratio (EM) for Fine Tuned DeepSeek Coder 1.3B V2: 0.02


# Evaluation Metrics DataFrame

In [ ]:
data = {'Base GEMMA-2B Model': [calculate_sacrebleu_score(final_df,'text','base_model_output'),calculate_bleu_score(final_df,'text','base_model_output'),calculate_exact_match_ratio(final_df,'text','base_model_output')],
        'Fine Tuned GEMMA-2B Model(400 rows)': [calculate_sacrebleu_score(final_df,'text','FT_model_output_400'),calculate_bleu_score(final_df,'text','FT_model_output_400'),calculate_exact_match_ratio(final_df,'text','FT_model_output_400')],
        'Fine Tuned GEMMA-2B Model(3000 rows)': [calculate_sacrebleu_score(final_df,'text','FT_model_output_3000'),calculate_bleu_score(final_df,'text','FT_model_output_3000'),calculate_exact_match_ratio(final_df,'text','FT_model_output_3000')],
        'Base DeepSeek Coder-1.3B Model': [calculate_sacrebleu_score(final_df,'text','Base_DeepSeek'),calculate_bleu_score(final_df,'text','Base_DeepSeek'),calculate_exact_match_ratio(final_df,'text','Base_DeepSeek')],
        'Fine Tuned DeepSeek Coder-1.3B Model V1': [calculate_sacrebleu_score(final_df,'text','FT_DeepSeek_V1'),calculate_bleu_score(final_df,'text','FT_DeepSeek_V1'),calculate_exact_match_ratio(final_df,'text','FT_DeepSeek_V1')],
        'Fine Tuned DeepSeek Coder-1.3B Model V2': [calculate_sacrebleu_score(final_df,'text','FT_DeepSeek_Coder_model_V2'),calculate_bleu_score(final_df,'text','FT_DeepSeek_Coder_model_V2'),calculate_exact_match_ratio(final_df,'text','FT_DeepSeek_Coder_model_V2')]
        }
df = pd.DataFrame.from_dict(data, orient='index', columns=['SACRE BLEU', 'BLEU SCORE', 'EM RATIO'])
df


,SACRE BLEU,BLEU SCORE,EM RATIO
Base GEMMA-2B Model,3.593964,0.035940,0.00
Fine Tuned GEMMA-2B Model(400 rows),70.381382,0.703814,0.14
Fine Tuned GEMMA-2B Model(3000 rows),70.381382,0.703814,0.14
Base DeepSeek Coder-1.3B Model,41.792700,0.417927,0.01
Fine Tuned DeepSeek Coder-1.3B Model V1,57.781606,0.577816,0.00
Fine Tuned DeepSeek Coder-1.3B Model V2,70.347961,0.703480,0.02


In [ ]:
file_path_colab = '/content/metrics.csv'
df.to_csv(file_path_colab,index=False)

># TODO: Do the above evaluation for all the used model